In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd_active_his
( 
  calday DATE,
  assortment STRING,
  assortment_class STRING,
  assortment_class_desc STRING,
  store_id STRING,
  vendor_id STRING,
  product_id STRING,
  start_date DATE,
  end_date DATE,
  store_delete INT,
  block_by_site INT,
  product_delete INT,
  block_by_article INT,
  block_by_store_product INT,
  block_by_assortment INT,
  block_by_assortment_ooa INT,
  block_by_ex INT,
  status STRING,
  status_ooa STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd_active_his
WHERE calday = DATE(current_timestamp() + INTERVAL 7 HOURS)
""")

In [0]:
spark.sql(
    f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd_active_his
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS calday,
    assortment,
    assortment_class,
    assortment_class_desc,
    store_id,
    vendor_id,
    product_id,
    start_date,
    CAST(end_date AS DATE) AS end_date,
    store_delete,
    block_by_site,
    product_delete,
    block_by_article,
    block_by_store_product,
    block_by_assortment,
    block_by_assortment_ooa,
    block_by_ex,
    status,
    status_ooa
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd_active
"""
)